# Forecasting Trees

This notebook intends on showcasing how the `TreeRegressor` class may be employed to predict hierarchical time series, based on the developed `TreeClass` objects

In [1]:
# Standard Libraries
import pandas as pd
import numpy as np
# Tree package
import hierarchicalearning.TreeClass as treepkg
import hierarchicalearning.HPL_utils as utils
import hierarchicalearning.TreeRegressor as regpkg

# Defining a simple (A, (B, C, D)) hierarchy
leaves = ['B', 'C', 'D']
periods = 100
random_data = np.random.rand(periods, 3)
tidx = pd.date_range('2022-01-01', periods=periods, freq='H')
df = pd.DataFrame(random_data, columns=leaves, index=tidx)

df_tree = dict()
for leaf_id in leaves:
    df_tree[leaf_id] = df.loc[:, [leaf_id]]
    df_tree[leaf_id].rename(columns={leaf_id: 'node_timeseries1'}, inplace=True)

# Creating a simple spatial tree
root = treepkg.Node(id='A',
                    parent=None,
                    children=leaves)
nodes = {root}
for leaf_id in leaves:
    leaf = treepkg.Node(id=leaf_id,
                        parent='A',
                        children=None)
    nodes.add(leaf)
tree = treepkg.Tree(nodes, dimension='spatial')
tree.create_spatial_hierarchy(df_tree, columns2aggr=['node_timeseries1'])

Loading utils...
done!


### Feature selection

In [2]:
#import statsmodels.api as sm
feature_toshift = {}
target = 'node_timeseries1'
threshold = 0.25

for node in tree.df:
    pac_largest_timedeltas = utils.get_pac_largest_timedeltas(tree.df[node][target], n=3, threshold=threshold)
    feature_toshift[node] = {target: pac_largest_timedeltas}

feature_toshift

No partial auto-correlation was found respecting the defined threshold. Threshold is re-adapted to highest PAC value:  0.20517570642764643
No partial auto-correlation was found respecting the defined threshold. Threshold is re-adapted to highest PAC value:  0.21278353230183344
No partial auto-correlation was found respecting the defined threshold. Threshold is re-adapted to highest PAC value:  0.20362413079810351
No partial auto-correlation was found respecting the defined threshold. Threshold is re-adapted to highest PAC value:  0.22054681064734785


{'B': {'node_timeseries1': ['P0DT18H0M0S']},
 'C': {'node_timeseries1': ['P0DT7H0M0S']},
 'D': {'node_timeseries1': ['P0DT18H0M0S']},
 'A': {'node_timeseries1': ['P0DT12H0M0S']}}

In [3]:
target = 'node_timeseries1'
### Defining target, features and feature shifts
target = [target]
features = dict()
for n in tree.leaves_label:
    features[n] = list(tree.df[n].columns)
    features[n].remove(target[0])

## Hierarchical Learning

In [4]:
# hierarchical forecasting methods:
hierarchical_forecasting_methods = ['multitask', 'OLS', 'BU', 'STR', 'hVAR', 'sVAR', 'COV', 'kCOV']
hierarchical_forecasting_method = 'multitask'
# reconciliation methods: 'None', 'OLS', 'BU', 'STR', 'hVAR', 'sVAR', 'COV', 'kCOV'
hierarchical_reconciliation_method = 'None'

hreg = regpkg.H_Regressor(tree, target_in=target, features_in=features,
                          forecasting_method=hierarchical_forecasting_method,
                          reconciliation_method=hierarchical_reconciliation_method,
                          alpha=0.75)

Here we redefine the features of the regressor with the method `feature_engineering_wrapper`.

In [5]:
hreg.feature_engineering_wrapper(feature_toshift)

engineering features of node:  B
engineering features of node:  C
engineering features of node:  D


In [6]:
# Obtain training and testing sets
from sklearn.preprocessing import MinMaxScaler, StandardScaler
nb_splits = 10
X_train, X_test, y_train, y_test, y_test_allflats = hreg.split_scaled(nb_splits=nb_splits, toscale=True,
                                                                      scaler=StandardScaler(),
                                                                      only_leaves_X=True)

Splitting and scaling training/testing sets, iteration  0  over  10
Splitting and scaling training/testing sets, iteration  1  over  10
Splitting and scaling training/testing sets, iteration  2  over  10
Splitting and scaling training/testing sets, iteration  3  over  10
Splitting and scaling training/testing sets, iteration  4  over  10
Splitting and scaling training/testing sets, iteration  5  over  10
Splitting and scaling training/testing sets, iteration  6  over  10
Splitting and scaling training/testing sets, iteration  7  over  10
Splitting and scaling training/testing sets, iteration  8  over  10
Splitting and scaling training/testing sets, iteration  9  over  10


In [8]:
# Create the network
hreg.create_ANN_network()

# Outputs to save
y_forecasted = []

## Train the model
for i in range(nb_splits):
    print(i)
    # For the first round, we initialise the G matrix as an OLS method - since we do not have prediction error estimates yet
    hreg.scaler_attribute_update(divider=np.transpose(hreg.target_scalers['divider'][i]),
                                    shift=np.transpose(hreg.target_scalers['shift'][i]))
    if i == 0:
        hreg.coherency_constraint(y_diff=None)
        hreg.yhat_initialization(columns_in=target)
    # We train the DANN
    hreg.regressor.fit(X_train[i],
                        y_train[i],
                        epochs=500,
                        batch_size=24,
                        verbose=0,
                        shuffle=False)

    # We obtain the y_hat prediction over the test set
    y_hat = hreg.regressor.predict(X_test[i])
    # Save y_hat to a hierarchical df format
    y_hat_unscaled = hreg.inverse_scale_y(y_hat)
    hreg.yhat = hreg.tree.reshape_flat2tree(flat_all_inputs=y_test_allflats[i], flat_data_in=y_hat_unscaled,
                                            tree_df_in=hreg.yhat)

    ## Update the coherency constraint in the loss function
    y_diff = np.transpose(y_test[i]) - np.transpose(y_hat)
    hreg.coherency_constraint(y_diff=y_diff)
    # We recompile the regressor to update the new loss function
    # see - https://stackoverflow.com/questions/60996892/how-to-replace-loss-function-during-training-tensorflow-keras
    loss_fct = hreg.coherency_loss_function_mse if hreg.hierarchical_forecasting_method != 'multitask' \
        else hreg.independent_loss_function_mse
    hreg.regressor.compile(loss=loss_fct,
                           optimizer='Adam',
                           metrics=['mae', 'mse'])


    # Inverse scaling y vectors for reconciliation
    y_test_unscaled = hreg.inverse_scale_y(y_test[i])
    y_hatrain = hreg.regressor.predict(X_train[i])
    y_hatrain_unscaled = hreg.inverse_scale_y(y_hatrain)
    y_train_unscaled = hreg.inverse_scale_y(y_train[i])
    y_diff = np.transpose(y_train_unscaled) - np.transpose(y_hatrain_unscaled)

    # A-posteriori hard-constrained reconciliation
    y_tild = hreg.hard_ctr_reconciliation(y_diff=y_diff, y_hat=y_hat_unscaled,
                                          method=hierarchical_reconciliation_method)
    y_forecasted.append(y_tild)

0
1
2
3
4
5
6
7
8
9
